# Global import


In [1]:
from pathlib import Path

from typing import Optional

import tiktoken

from openai import OpenAI

from app.cost.cost import CostTracker

from app.proposal_object.schemas import Devis

from app.proposal_object.path import ProposalObjectPaths

from app.proposal_object.proposal_object import ProposalObjectHandler

from config.logger_config import logger


/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(row[0], str) and row[0].strip() == "Lot :":
/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  isinstance(row[2], str)
/Users/remillieux/Documents/Kleek/OCR/app/proposal_object/schemas.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

# API Keys


In [2]:
AZURE_DI_ENDPOINT = "https://cleed-di.cognitiveservices.azure.com/"

AZURE_DI_KEY = "ae3ff9603ad64f81bf052eeee5a1ffae"


# Cost tracker


In [3]:
cost_tracker = CostTracker()


# Proposal


In [ ]:
### Hard ###
# FILE_NAME = "Devis DE2502-580 - LABOURDETTE - "


### Medium ###
FILE_NAME = "Lot Electricité - JUILLARD - Devis_n°OF9206"

# FILE_NAME = "Devis - CHAINE CHAINE DES ARTISANS COTE SUD - DEVIS LARTIZIEN   1"

# FILE_NAME = "Devis_'OF9189'"

# FILE_NAME = "Devis DE2412-461 - GOURDON - "

### Easy ###
# FILE_NAME = "Devis - CHAINE CHAINE DES ARTISANS COTE SUD - Vouland   Seignosse"

proposal_path = ProposalObjectPaths(file_name=FILE_NAME)


# Utils


In [5]:
def token_counter(text: str) -> int:
    """Count the nb of token in a text"""

    encoding = tiktoken.get_encoding("cl100k_base")

    nb_token: int = len(encoding.encode(text))

    return nb_token


# OpenAI Vision


In [6]:
PROPOSAL_SECTION_ANALYZER_SYSTEM_PROMPT = """Vous êtes le meilleur analyseur de devis du monde. Vous devez analyser un devis au format HTML pour déterminer la structure des catégories de produits de celui-ci, c'est-à-dire les différentes sections et sous-sections des produits du devis. Vous devez vérifier que le prix total de la catégorie correspond bien au prix des produits listés dans la catégorie. Respectez la structure originale du devis, ne tentez pas de fusionner des lignes."""

client = OpenAI()

file = client.files.create(
    file=open(proposal_path.proposal_path, "rb"), purpose="user_data"
)

completion = client.beta.chat.completions.parse(
    model="gpt-4.1",
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": PROPOSAL_SECTION_ANALYZER_SYSTEM_PROMPT,
                },
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "file",
                    "file": {
                        "file_id": file.id,
                    },
                }
            ],
        },
    ],
    response_format=Devis,
    temperature=0.0,
)

proposal: Optional[Devis] = completion.choices[0].message.parsed

if proposal is None:
    logger.error("PROPOSAL => No structured proposal found.")


proposal_json = proposal.model_dump_json(indent=2)

a_out = Path(proposal_path.proposal_object_predicted_path)

a_out.parent.mkdir(parents=True, exist_ok=True)

a_out.write_text(proposal_json)

print(f"✅ OBJECT prédit écrit dans {a_out.resolve()}")

print(f"Cost : {cost_tracker.cost.model_dump_json(indent=2)}")


✅ OBJECT prédit écrit dans /Users/remillieux/Documents/Kleek/OCR/data/proposal_object_predicted/Lot Electricité - JUILLARD - Devis_n°OF9206.json
Cost : {
  "cost": 0.0,
  "cost_openai": {
    "cost_openai_input": 0.0,
    "cost_openai_output": 0.0,
    "cost_embeddings": 0.0,
    "cost_openai_total": 0.0,
    "nb_query": 0,
    "queries": []
  }
}


# Proposal object


In [7]:
ProposalObjectHandler(proposal_path=proposal_path).analyze_proposal_object()


2025-06-12 13:33:44 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_text.49]:                             ✅ PROPOSAL TEXT write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_text/Lot Electricité - JUILLARD - Devis_n°OF9206.txt
2025-06-12 13:33:44 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_pdf.65]:                              ✅ PROPOSAL PDF write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_pdf_object/Lot Electricité - JUILLARD - Devis_n°OF9206.pdf
2025-06-12 13:33:44 INFO -> [app/proposal_object/proposal_object.py.proposal_object.convert_to_proposal_lines.94]:                   ✅ PROPOSAL LINES write in /Users/remillieux/Documents/Kleek/OCR/data/proposal_object_line/Lot Electricité - JUILLARD - Devis_n°OF9206.txt
2025-06-12 13:33:44 INFO -> [app/proposal_object/proposal_object.py.proposal_object.validate_proposal_object.116]:                   {
  "computed_total_ht": 12487.42,
  "computed_total_tva": 2497.5

arraybcimunnm = [
    "|     PACK    |   TVA   |                                                                                                                                                                  LABEL                                                                                                                                                                  |                  DESCRIPTION                   | QTY | UNIT | UNIT_PRICE_HT | TOTAL_HT |",
    "|-------------|---------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|-----|------|---------------|----------|",
    "| ELECTRICITE | TVA 20% |                                                